# Python Typing and Async

## Learning Objectives

- Write clear type hints for functions and collections
- Define interfaces using Protocol
- Compare dataclasses and Pydantic models
- Use async and await correctly
- Understand async boundaries in API vs Celery tasks

---

## 1. Type Hints Basics

In [ ]:
from typing import Optional

def normalize_query(text: str, limit: int = 128) -> str:
    cleaned = text.strip()
    return cleaned[:limit]

def find_user(user_id: str) -> Optional[dict[str, str]]:
    if user_id == 'missing':
        return None
    return {'id': user_id, 'name': 'Alice'}

print(normalize_query('  hello  ', 5))
print(find_user('42'))
print(find_user('missing'))

## 2. Protocol for Interfaces

In [ ]:
from typing import Protocol

class Document(Protocol):
    id: str
    title: str
    content: str

class DocumentRepo(Protocol):
    def save(self, doc: Document) -> Document:
        ...

    def get(self, doc_id: str) -> Document | None:
        ...

print('Protocol interfaces defined')

## 3. Dataclasses for Domain Models

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Doc:
    id: str
    title: str
    content: str

doc = Doc(id='1', title='Intro', content='Text')
print(doc)

## 4. Async Basics

In [ ]:
import asyncio

async def fetch_doc(doc_id: str) -> str:
    await asyncio.sleep(0.1)
    return f'doc:{doc_id}'

async def main() -> None:
    result = await fetch_doc('1')
    print(result)

asyncio.run(main())

## 5. Async Fan-Out

In [ ]:
import asyncio

async def fetch_many(ids: list[str]) -> list[str]:
    tasks = [fetch_doc(doc_id) for doc_id in ids]
    return await asyncio.gather(*tasks)

async def main_many() -> None:
    results = await fetch_many(['1', '2', '3'])
    print(results)

asyncio.run(main_many())

## 6. Async Boundaries in Celery

In [ ]:
import asyncio

async def async_job(doc_id: str) -> str:
    await asyncio.sleep(0.05)
    return doc_id

def sync_task(doc_id: str) -> str:
    # Celery tasks are sync by default, so we run async code inside.
    return asyncio.run(async_job(doc_id))

print(sync_task('99'))

## Summary

- Type hints make interfaces explicit
- Protocols are a clean way to define adapter contracts
- Dataclasses are great for domain models
- Async is for I/O, not CPU-heavy work
- Celery tasks stay sync and can run async helpers